In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
!unzip '/content/drive/My Drive/ens492/ISIC_2019_Training_Input.zip' -d /content/train_dataset
!unzip '/content/drive/My Drive/ens492/ISIC_2019_Test_Input.zip' -d /content/test_dataset

!pip install focal-loss-torch
"""
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

!pip install focal-loss-torch


import os
os.environ["WANDB_DISABLED"] = "true"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
! nvcc --version
!nvidia-smi -L
device

Mounted at /content/gdrive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successf

device(type='cuda')

In [ ]:
!unzip '/content/gdrive/My Drive/Colab Notebooks/image_processing/archive.zip'     -d /content/train_dataset
!unzip '/content/gdrive/My Drive/Colab Notebooks/image_processing/archive.zip'     -d /content/test_dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057952.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057954.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057955.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057956.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057957.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057961.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057965.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057967.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057968.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057971.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057972.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test_Input/ISIC_0057976.jpg  
  inflating: /content/test_dataset/ISIC_2019_Test

In [ ]:
# General Libraries
import os
import sys
import numpy as np
import pandas as pd
import random
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import torchvision.transforms.functional as TF
from collections import Counter

# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# PyTorch Vision Libraries
import torchvision.utils
from torchvision import models, datasets, transforms

# Machine Learning & Metrics
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Additional Libraries
import timm
from tqdm import tqdm
from focal_loss.focal_loss import FocalLoss

# Visualization
%matplotlib inline


# Preprocessing

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Random Seed Set
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# If using CUDA, ensure deterministic behavior as well
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # for multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Data paths
data_dir =   './content/drive/My Drive/ens492'
train_dir =  './content/train_dataset/ISIC_2019_Training_Input'
test_dir =   './content/test_dataset/ISIC_2019_Test_Input'
train_ground_truth = 'ISIC_2019_Training_GroundTruth.csv'
test_ground_truth = 'ISIC_2019_Test_GroundTruth.csv'
# Load and process ground truth
train_ground_truth_df = pd.read_csv(os.path.join(data_dir, train_ground_truth))
test_ground_truth_df = pd.read_csv(os.path.join(data_dir, test_ground_truth))

# Process training data - exclude 'image' column and create single_label
train_labels = train_ground_truth_df.iloc[:, 1:9]  # Exclude 'image' and 'UNK' columns
train_ground_truth_df['single_label'] = train_labels.values.argmax(axis=1)

# Process test data - exclude 'image', 'score_weight', and 'validation_weight' columns
test_labels = test_ground_truth_df.iloc[:, 1:10]  # Include UNK class
test_ground_truth_df['single_label'] = test_labels.values.argmax(axis=1)

# Ensure patient ID-based stratification to avoid data leakage
train_ground_truth_df['patient_id'] = train_ground_truth_df['image']

# Stratified split by patient ID
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# Generate train/val indices
for train_idx, val_idx in sgkf.split(train_ground_truth_df, train_ground_truth_df['single_label'], groups=train_ground_truth_df['patient_id']):
    train_df = train_ground_truth_df.iloc[train_idx].reset_index(drop=True)
    val_df = train_ground_truth_df.iloc[val_idx].reset_index(drop=True)
    break  # Take the first fold as train/val split

# Drop patient_id column as it's no longer needed
train_df = train_df.drop(columns=['patient_id'])
val_df = val_df.drop(columns=['patient_id'])

# Augmentations
class CircularMask:
    def __call__(self, image):
        w, h = image.size
        # Create mask
        mask = Image.new("L", (w, h), 0)
        draw = ImageDraw.Draw(mask)
        # Draw white circle in the center (adjust scale here if needed)
        radius = min(w, h) // 2
        draw.ellipse((w//2 - radius, h//2 - radius, w//2 + radius, h//2 + radius), fill=255)

        # Convert image to RGBA for masking
        image = image.convert("RGBA")
        black_bg = Image.new("RGBA", image.size, (0, 0, 0, 255))
        image = Image.composite(image, black_bg, mask)

        # Convert back to RGB
        return image.convert("RGB")

class CenterSquareCrop:
    def __call__(self, image):
        w, h = image.size
        min_dim = min(w, h)
        left = (w - min_dim) // 2
        top = (h - min_dim) // 2
        right = left + min_dim
        bottom = top + min_dim
        return image.crop((left, top, right, bottom))


train_transform = transforms.Compose([
    CenterSquareCrop(),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.RandomRotation(degrees=45), # was 90
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),

    transforms.RandomApply([
        transforms.RandomAffine(
            degrees=0.5,
            translate=(0.1, 0.1),
            scale=(1.0, 1.05),
            fill=(200, 200, 200)
        )
    ], p=0.5),


    transforms.RandomAffine(degrees=0.5, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # slightly more conservative
    #CircularMask(),  # custom transform here
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomErasing(
        p=0.5,
        scale=(16*16 / (224*224), 16*16 / (224*224)),
        ratio=(1.0, 1.0),
        value='random'
    ),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    CenterSquareCrop(),  # or CenterSquareCrop
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Modify the dataset class to handle both training and test data
class ISICDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image'] + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path)

        if self.is_test:
            # For test data, include UNK class
            label_cols = self.dataframe.iloc[:, 1:10]  # MEL to UNK
        else:
            # For training data, exclude UNK class
            label_cols = self.dataframe.iloc[:, 1:9]  # MEL to SCC

        label = self.dataframe.iloc[idx]['single_label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Create datasets and dataloaders
BATCH_SIZE = 512
train_dataset = ISICDataset(train_df, train_dir, train_transform, is_test=False)
val_dataset = ISICDataset(val_df, train_dir, val_transform, is_test=False)
test_dataset = ISICDataset(test_ground_truth_df, test_dir, val_transform, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

FileNotFoundError: [Errno 2] No such file or directory: './content/drive/My Drive/ens492/ISIC_2019_Training_GroundTruth.csv'

## Augmentation Results:

In [ ]:
# Helper to undo normalization
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    return torch.clamp(tensor * std[:, None, None] + mean[:, None, None], 0, 1)

# Pick 3 random indices from the train_df
sample_indices = random.sample(range(len(train_df)), 3)

# Create figure
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))

for row, idx in enumerate(sample_indices):
    img_name = train_df.iloc[idx]['image'] + '.jpg'
    img_path = os.path.join(train_dir, img_name)

    # Load original
    original_image = Image.open(img_path).convert("RGB")

    # Augmented version
    augmented_tensor = train_transform(original_image)
    augmented_image = denormalize(augmented_tensor)
    augmented_image = TF.to_pil_image(augmented_image)

    # Plot original
    axes[row, 0].imshow(original_image)
    axes[row, 0].set_title(f'Original #{idx}')
    axes[row, 0].axis('off')

    # Plot augmented
    axes[row, 1].imshow(augmented_image)
    axes[row, 1].set_title(f'Augmented #{idx}')
    axes[row, 1].axis('off')

plt.tight_layout()
plt.show()


## Training Routine
- StratifiedGroupKFold for Train/Val Splitting
- AdamW as Optimizer
- Focal Loss as Loss Function
- GradScaler for Mixed Precision Training (halves training time)

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt

# === Model & Training Setup ===
num_classes = 9
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
model = model.to(device)

criterion = FocalLoss(gamma=2.0, reduction='mean')  # use focal_loss_torch

# Optimizer
base_lr = 1e-5
optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=1e-3)

# Mixed Precision
scaler = GradScaler()

# === Train Function ===
def train_one_epoch(model, dataloader, optimizer):
    model.train()
    correct, total = 0, 0
    total_loss = 0.0  # Track total loss for the epoch

    for inputs, targets in tqdm(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(inputs)
            outputs = torch.softmax(outputs, dim=1)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * targets.size(0)  # accumulate total loss

        with torch.no_grad():
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

    avg_loss = total_loss / total  # Calculate average loss for the epoch
    return 100. * correct / total, avg_loss  # Return accuracy and average loss

# === Evaluate Function ===
def evaluate(model, dataloader, criterion, is_test=False):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels, all_probs = [], [], []
    total_loss = 0.0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=-1)
            loss = criterion(probs, targets)
            total_loss += loss.item() * targets.size(0)

            if is_test:
                max_probs, predicted = probs.max(1)
                unk_probs = 1 - max_probs
                predicted = torch.where(unk_probs > 0.5,
                                        torch.tensor(num_classes-1).to(device),
                                        predicted)
            else:
                _, predicted = probs.max(1)

            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    acc = 100. * correct / total
    return acc, avg_loss, macro_f1, all_preds, all_labels, all_probs

# === Dataloaders ===
#test_dataset = ISICDataset(test_ground_truth_df, test_dir, val_transform)
#test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# === Training Loop ===
num_epochs = 25
best_val_f1 = 0
best_val_loss = float('inf')  # Start with infinity for the best validation loss

# Tracking metrics
train_accs, val_accs, train_losses, val_losses, val_f1s = [], [], [], [], []

print("Starting training...")

for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch+1}/{num_epochs}]')
    train_acc, train_loss = train_one_epoch(model, train_loader, optimizer)
    val_acc, val_loss, val_f1, val_preds, val_labels, val_probs = evaluate(model, val_loader, criterion)

    # Log metrics
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_f1s.append(val_f1)

    print(f'Train Accuracy      : {train_acc:.2f}%')
    print(f'Validation Accuracy : {val_acc:.2f}%')
    print(f'Train Loss          : {train_loss:.4f}')
    print(f'Validation Loss     : {val_loss:.4f}')
    print(f'Validation Macro F1 : {val_f1:.4f}')

    """
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'best_vit_model.pth')
        print(f'✅ Model saved! Best Macro F1: {best_val_f1:.4f}')
    """

    # Save the best model based on lowest validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_vit_model_lowest_loss.pth')
        print(f'✅ Model saved! Best Validation Loss: {best_val_loss:.4f}')

Saved model results:

Train Accuracy      : 96.90%

Validation Accuracy : 86.83%

Train Loss          : 0.0290

Validation Loss     : 0.2412

Validation Macro F1 : 0.8192

✅ Model saved! Best Macro F1: 0.8192

## Training Plots:

In [ ]:

# === Plotting After Training ===
epochs = list(range(1, num_epochs + 1))

plt.figure(figsize=(15, 5))

# Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, train_accs, label='Train Acc')
plt.plot(epochs, val_accs, label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, train_losses, label='Train Loss', color='blue')
plt.plot(epochs, val_losses, label='Val Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()

# Macro F1
plt.subplot(1, 3, 3)
plt.plot(epochs, val_f1s, label='Val Macro F1', color='green')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Validation Macro F1')
plt.legend()

plt.tight_layout()
plt.show()


## Validation Conf Matrix:

In [ ]:
# Load the trained model
model.load_state_dict(torch.load('best_vit_model_lowest_loss.pth'))

# Assuming 'val_loader' is your DataLoader for the validation dataset
# If you don't have 'val_loader', replace it with your actual validation DataLoader
# Example:
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

def evaluate(model, dataloader, criterion=None, is_test=False):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    all_probs = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=-1)

            if is_test:
                max_probs, predicted = probs.max(1)
                unk_probs = 1 - max_probs
                predicted = torch.where(unk_probs > 0.5,
                                        torch.tensor(num_classes-1).to(device),
                                        predicted)
            else:
                _, predicted = probs.max(1)

            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    if criterion is not None:
        return 100. * correct / total, all_preds, all_labels, all_probs
    else:
        return 100. * correct / total, all_preds, all_labels, all_probs

# Evaluate the model on the validation set
final_val_acc, val_preds, val_labels, _ = evaluate(model, val_loader, criterion=None)

# Print final validation accuracy
print(f'Final Validation Accuracy: {final_val_acc:.2f}%')

# Compute confusion matrix
cm = confusion_matrix(val_labels, val_preds)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix for Validation Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

Val Acc was: 86.19

## Last Training with Val + Train:

In [ ]:
# Load the best model saved during training
model.load_state_dict(torch.load('best_vit_model_lowest_loss.pth'))

# === Combine the Training and Validation Data ===
combined_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])  # Combine train and val datasets
combined_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# === Fine-Tune on Combined Dataset ===
num_fine_tune_epochs = 5  # Fine-tune for a few epochs with the combined dataset
for epoch in range(num_fine_tune_epochs):
    print(f'\nFine-tuning Epoch [{epoch+1}/{num_fine_tune_epochs}]')
    train_acc, train_loss = train_one_epoch(model, combined_loader, optimizer)
    print(f'Train Accuracy: {train_acc:.2f}%')
    print(f'Train Loss: {train_loss:.4f}')

torch.save(model.state_dict(), 'best_vit_model_after_val.pth')
print(f'✅ Model saved! Best Macro F1: {best_val_f1:.4f}')

# Testing:

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

model.load_state_dict(torch.load('best_vit_model_after_val.pth'))
test_dir = './content/test_dataset/ISIC_2019_Test_Input'
test_dataset = ISICDataset(test_ground_truth_df, test_dir, val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

def evaluate(model, dataloader, is_test=False):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    all_probs = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probs = outputs.softmax(dim=-1)

            if is_test:
                # For test set, assign (1-max_prob) to UNK class
                max_probs, predicted = probs.max(1)
                unk_probs = 1 - max_probs
                # Adjust predictions based on confidence threshold
                predicted = torch.where(unk_probs > 0.5,
                                      torch.tensor(num_classes-1).to(device),  # UNK class
                                      predicted)
            else:
                _, predicted = probs.max(1)

            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    return 100. * correct / total, all_preds, all_labels, all_probs

# After training loop, add test evaluation
print("\nEvaluating on test set...")
test_acc, test_preds, test_labels, test_probs = evaluate(model, test_loader, is_test=True)
print(f'Test Accuracy: {test_acc:.2f}%')

# Calculate and display confusion matrix for test set
cm_test = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
plt.title('Test Set Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Calculate precision, recall, F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(test_labels, test_preds, average='weighted')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

# Print classification report
print("\nClassification Report:")
print(classification_report(test_labels, test_preds))

# Save test predictions
test_results = pd.DataFrame({
    'image': test_ground_truth_df['image'],
    'predicted_class': test_preds,
    'true_class': test_labels
})

test_results.to_csv('test_predictions.csv', index=False)

Test acc was: 55.15

## Evaluation without UNK

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def evaluate_ignore_unk(model, dataloader):
    """
    Evaluates the model on test set, ignoring samples that are labeled as UNK class.
    Works with a model trained on 9 classes but only evaluates on the first 8 classes.
    """
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    all_probs = []
    misclassified_images = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Skip samples labeled as UNK (class 8)
            mask = targets != 8
            if not mask.any():
                continue

            # Filter out UNK samples
            filtered_inputs = inputs[mask]
            filtered_targets = targets[mask]

            if len(filtered_inputs) == 0:
                continue

            outputs = model(filtered_inputs)
            probs = outputs.softmax(dim=-1)

            # Only consider probabilities for known classes (0-7)
            known_probs = probs[:, :8]
            _, predicted = known_probs.max(1)

            correct += predicted.eq(filtered_targets).sum().item()
            total += filtered_targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(filtered_targets.cpu().numpy())
            all_probs.extend(known_probs.cpu().numpy())

            # Collect misclassified images for visualization
            for i in range(len(filtered_targets)):
                if predicted[i] != filtered_targets[i]:
                    misclassified_images.append((filtered_inputs[i].cpu(), predicted[i].cpu(), filtered_targets[i].cpu()))

    accuracy = 100. * correct / total if total > 0 else 0
    return accuracy, all_preds, all_labels, all_probs, misclassified_images

# After training loop, evaluate ignoring UNK class
print("\nEvaluating on test set (ignoring UNK class)...")
test_acc, test_preds, test_labels, test_probs, misclassified_images = evaluate_ignore_unk(model, test_loader)
print(f'Test Accuracy (excluding UNK): {test_acc:.2f}%')

# Calculate F1 Score (using weighted average)
f1 = f1_score(test_labels, test_preds, average='weighted')
print(f'F1 Score (excluding UNK): {f1:.4f}')

# Calculate and display confusion matrix for test set (8 classes)
cm_test = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
plt.title('Test Set Confusion Matrix (excluding UNK)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Display some misclassified images
print(f"Displaying first {len(misclassified_images)} misclassified images:")
num_images = 5  # Change this to display more or fewer images
plt.figure(figsize=(15, 10))

for i, (image, pred, true) in enumerate(misclassified_images[:num_images]):
    plt.subplot(1, num_images, i + 1)

    # If the image tensor is in format [C, H, W], permute it to [H, W, C]
    image = image.permute(1, 2, 0).cpu().numpy()  # Convert to HxWxC format (if necessary)

    # Ensure the image is in the right range for plotting
    if image.min() < 0 or image.max() > 1:
        image = (image - image.min()) / (image.max() - image.min())  # Normalize to [0, 1]

    plt.imshow(image)
    plt.title(f"Pred: {pred}, True: {true}")
    plt.axis('off')

plt.show()


# Save test predictions (excluding UNK)
test_results = pd.DataFrame({
    'image': [test_ground_truth_df['image'][i] for i, label in enumerate(test_labels) if label != 8],
    'predicted_class': test_preds,
    'true_class': test_labels
})
test_results.to_csv('test_predictions_no_unk.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ISIC 2019 Class Names
class_names = [
    'Melanoma',
    'Nevus',
    'Basal Cell Carcinoma',
    'Actinic Keratosis',
    'Benign Keratosis-like Lesions',
    'Dermatofibroma',
    'Vascular Lesions',
    'Squamous Cell Carcinoma'
]

# Collect one misclassified image for each class
misclassified_by_class = {class_name: None for class_name in class_names}

for image, pred, true in misclassified_images:
    true_class_name = class_names[true]
    if misclassified_by_class[true_class_name] is None:
        misclassified_by_class[true_class_name] = (image, pred, true)
    # Stop once we have one misclassified image for each class
    if all(misclassified_by_class[class_name] is not None for class_name in class_names):
        break

# Display one misclassified image per class (2x4 grid)
plt.figure(figsize=(15, 10))

for i, (class_name, (image, pred, true)) in enumerate(misclassified_by_class.items()):
    plt.subplot(2, 4, i + 1)

    # If the image tensor is in format [C, H, W], permute it to [H, W, C]
    image = image.permute(1, 2, 0).cpu().numpy()  # Convert to HxWxC format (if necessary)

    # Ensure the image is in the right range for plotting
    if image.min() < 0 or image.max() > 1:
        image = (image - image.min()) / (image.max() - image.min())  # Normalize to [0, 1]

    plt.imshow(image)
    plt.title(f"Pred: {class_names[pred]}\nTrue: {class_names[true]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
class ViTGradCAM:
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.gradients = None
        self.attentions = None

        # Register hooks to capture attention and gradients
        def save_attention(module, input, output):
            self.attentions = output  # Capture attention maps (output)

        def save_gradient(module, grad_input, grad_output):
            self.gradients = grad_output[0]  # Capture gradients from the output

        # Hook into the multi-head attention mechanism (Self-Attention)
        self.hooks = []
        for block in model.blocks:
            # Register forward hook for attention output
            hook = block.attn.register_forward_hook(save_attention)
            self.hooks.append(hook)

            # Register backward hook for gradients
            hook_grad = block.attn.register_backward_hook(save_gradient)
            self.hooks.append(hook_grad)

    def generate_cam(self, input_image, target_class):
        # Forward pass through the model
        output = self.model(input_image)
        self.model.zero_grad()

        # Backward pass for target class
        class_loss = output[0, target_class]
        class_loss.backward()

        # Get attention maps and gradients
        attentions = self.attentions.cpu().data.numpy()  # Shape: [batch, num_heads, num_patches, num_patches]
        gradients = self.gradients.cpu().data.numpy()  # Shape: [batch, num_heads, num_patches, num_patches]

        # Check the shapes of the gradients and attentions
        print(f"Gradients shape: {gradients.shape}")
        print(f"Attentions shape: {attentions.shape}")

        # Handle the case where gradients have 3 dimensions instead of 4
        if len(gradients.shape) == 3:  # If gradients don't have the expected 4th dimension
            gradients = np.expand_dims(gradients, axis=-1)  # Add the missing dimension

        # Weight the attention maps by the gradients
        weighted_attn = np.mean(gradients, axis=(2, 3), keepdims=True)  # Shape: [batch, num_heads, 1, 1]
        cam = np.sum(weighted_attn * attentions, axis=1)  # Sum across heads

        # CAM has shape [batch, 1, num_patches, num_patches], normalize it
        cam = np.maximum(cam, 0)
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)

        # Resize CAM to match input image size
        cam = np.resize(cam, (input_image.shape[2], input_image.shape[3]))

        return cam


In [ ]:
def visualize_grad_cam(image, cam, title="Grad-CAM"):
    # Convert the image tensor to a numpy array (if needed)
    image = image.permute(1, 2, 0).cpu().numpy()
    if image.min() < 0 or image.max() > 1:
        image = (image - image.min()) / (image.max() - image.min())  # Normalize image to [0, 1]

    # Resize CAM to match image size
    heatmap = np.uint8(255 * cam)  # Normalize CAM for visualization
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply a color map to the heatmap
    heatmap = np.float32(heatmap) / 255

    # Overlay heatmap on the original image
    overlay = heatmap + np.float32(image)
    overlay = overlay / np.max(overlay)

    # Plot the result
    plt.figure(figsize=(10, 10))
    plt.imshow(overlay)
    plt.title(title)
    plt.axis('off')
    plt.show()


# Assuming `model` is your pre-trained Vision Transformer and you have a misclassified image
grad_cam = ViTGradCAM(model)

# Get a misclassified image from your misclassified images list
image, pred, true = misclassified_images[20]  # Use the first image for visualization
image = image.unsqueeze(0).to(device)  # Add batch dimension
cam = grad_cam.generate_cam(image, pred.item())

# Visualize Grad-CAM on the misclassified image
visualize_grad_cam(image.squeeze(), cam, title=f"Pred: {pred.item()}, True: {true.item()}")
